In [ ]:
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install ipython-sql prettytable
%load_ext sql

import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

!pip install -q pandas
%sql sqlite:///my_data1.db

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp311-cp311-linux_x86_64.whl size=1193602 sha256=2d3df01aa329f62c8763b3e65c06b1c3fe8ee931c8f083b0ee23c5af720f09a9
  Stored in directory: /root/.cache/pip/wheels/3a/7c/1e/12404784a68083eb969f877a1808a1847bab897684b56ddc55
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.40
    Uninstalling SQLAlchemy-2.0.40:
      Successfully uninstalled SQLAlchemy-2.0.40
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.23 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.3.9 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [ ]:
#DROP THE TABLE IF EXISTS
%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [ ]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [ ]:
#Task 1: display the names of the unique launch sites in the space mission
full_table = %sql select * from SPACEXTABLE
print(full_table)

 * sqlite:///my_data1.db
Done.
+------------+------------+-----------------+--------------+---------------------------------------------------------------+-------------------+-------------+-----------------------------------------------------------+----------------------------------+------------------------+
|    Date    | Time (UTC) | Booster_Version | Launch_Site  |                            Payload                            | PAYLOAD_MASS__KG_ |    Orbit    |                          Customer                         |         Mission_Outcome          |    Landing_Outcome     |
+------------+------------+-----------------+--------------+---------------------------------------------------------------+-------------------+-------------+-----------------------------------------------------------+----------------------------------+------------------------+
| 2010-06-04 |  18:45:00  |  F9 v1.0  B0003 | CCAFS LC-40  |              Dragon Spacecraft Qualification Unit             |        

In [ ]:
unique_launchsite = %sql select distinct Launch_Site from SPACEXTABLE
print(unique_launchsite)

 * sqlite:///my_data1.db
Done.
+--------------+
| Launch_Site  |
+--------------+
| CCAFS LC-40  |
| VAFB SLC-4E  |
|  KSC LC-39A  |
| CCAFS SLC-40 |
+--------------+


In [ ]:
#Task 2 Display 5 records where launch sites begin with the string 'CCA'
task2 = %sql select * from SPACEXTABLE where Launch_Site Like 'CCA%' limit 5
print(task2)

 * sqlite:///my_data1.db
Done.
+------------+------------+-----------------+-------------+---------------------------------------------------------------+-------------------+-----------+-----------------+-----------------+---------------------+
|    Date    | Time (UTC) | Booster_Version | Launch_Site |                            Payload                            | PAYLOAD_MASS__KG_ |   Orbit   |     Customer    | Mission_Outcome |   Landing_Outcome   |
+------------+------------+-----------------+-------------+---------------------------------------------------------------+-------------------+-----------+-----------------+-----------------+---------------------+
| 2010-06-04 |  18:45:00  |  F9 v1.0  B0003 | CCAFS LC-40 |              Dragon Spacecraft Qualification Unit             |         0         |    LEO    |      SpaceX     |     Success     | Failure (parachute) |
| 2010-12-08 |  15:43:00  |  F9 v1.0  B0004 | CCAFS LC-40 | Dragon demo flight C1, two CubeSats, barrel of Brouer

In [ ]:
task_3 = %sql SELECT SUM(PAYLOAD_MASS__KG_) FROM SPACEXTABLE  WHERE Customer LIKE '%NASA (CRS)%';
print(task_3)

 * sqlite:///my_data1.db
Done.
+------------------------+
| SUM(PAYLOAD_MASS__KG_) |
+------------------------+
|         48213          |
+------------------------+


In [ ]:
#Task 4: Display average payload mass carried by booster version F9 v1.1
task_4 = %sql SELECT AVG(PAYLOAD_MASS__KG_) FROM SPACEXTABLE  WHERE Booster_Version = 'F9 v1.1';
print(task_4)

 * sqlite:///my_data1.db
Done.
+------------------------+
| AVG(PAYLOAD_MASS__KG_) |
+------------------------+
|         2928.4         |
+------------------------+


In [ ]:
#Task 5: List the date when the first successful landing outcome in ground pad was achieved
%sql SELECT MIN(Date) AS First_Successful_Landing_Date  FROM SPACEXTABLE  WHERE Landing_Outcome = 'Success (ground pad)';


 * sqlite:///my_data1.db
Done.


First_Successful_Landing_Date
2015-12-22


In [ ]:
#Task 6: List the names of boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql SELECT DISTINCT Booster_Version  FROM SPACEXTABLE  WHERE Landing_Outcome = 'Success (drone ship)'  AND PAYLOAD_MASS__KG_ > 4000  AND PAYLOAD_MASS__KG_ < 6000;


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [ ]:
#Task 7: List the total number of successful and failure mission outcomes
%sql SELECT Landing_Outcome, COUNT(*) AS Total_Count  FROM SPACEXTABLE  GROUP BY Landing_Outcome;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Total_Count
Controlled (ocean),5
Failure,3
Failure (drone ship),5
Failure (parachute),2
No attempt,21
No attempt,1
Precluded (drone ship),1
Success,38
Success (drone ship),14
Success (ground pad),9


In [ ]:
#Task 8: List all booster versions that have carried the maximum payload mass
%sql SELECT Booster_Version  FROM SPACEXTABLE  WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE);


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [ ]:
#Task 9: List records for months in year 2015
%sql SELECT substr(Date, 6, 2) AS Month, Booster_Version, Launch_Site, Landing_Outcome  FROM SPACEXTABLE  WHERE substr(Date, 1, 4) = '2015';


 * sqlite:///my_data1.db
Done.


Month,Booster_Version,Launch_Site,Landing_Outcome
01,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
02,F9 v1.1 B1013,CCAFS LC-40,Controlled (ocean)
03,F9 v1.1 B1014,CCAFS LC-40,No attempt
04,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)
04,F9 v1.1 B1016,CCAFS LC-40,No attempt
06,F9 v1.1 B1018,CCAFS LC-40,Precluded (drone ship)
12,F9 FT B1019,CCAFS LC-40,Success (ground pad)


In [ ]:
#Task 10: Rank landing outcomes between specific dates in descending order
%sql SELECT Landing_Outcome, COUNT(*) AS Outcome_Count  FROM SPACEXTABLE  WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'  GROUP BY Landing_Outcome  ORDER BY Outcome_Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
